In [6]:
# Imports & File Paths
import pandas as pd
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt

CLEAN_FILE = "../../MLBDATA/Processed/BatterData/ModelingBattingData.csv"

In [5]:
# Load Data & Preprocessing
df = pd.read_csv(CLEAN_FILE)

# Create PostBan flag
df['PostBan'] = (df['year'] >= 2023).astype(int)

# Exclude non-feature cols
exclude = [
    'player_id', 'last_name, first_name', 'year',
    'woba', 'cluster', 'Archetype'
]
features = [c for c in df.columns if c not in exclude]

df.head()

,"last_name, first_name",player_id,year,player_age,ab,pa,hit,single,double,triple,...,pull_percent,opposite_percent,batted_ball,groundballs_percent,flyballs_percent,popups_percent,sprint_speed,cluster,Archetype,PostBan
0,"McCutchen, Andrew",457705,2016,29,598,675,153,100,26,3,...,43.2,22.1,458,36.9,24.2,9.2,29.1,1,All-or-Nothing Fly-Ball Sluggers,0
1,"Turner, Justin",457759,2016,31,556,622,153,89,34,3,...,35.2,30.9,457,36.3,27.6,6.6,26.2,2,Power-Contact Hitters,0
2,"Santana, Carlos",467793,2016,30,582,688,151,83,31,3,...,48.0,16.8,488,43.0,21.5,9.8,25.9,1,All-or-Nothing Fly-Ball Sluggers,0
3,"Goldschmidt, Paul",502671,2016,28,579,704,172,112,33,3,...,38.2,26.5,437,47.4,19.0,5.5,27.3,2,Power-Contact Hitters,0
4,"Altuve, Jose",514888,2016,26,640,717,216,145,42,5,...,39.5,19.5,580,43.4,22.2,4.3,28.4,2,Power-Contact Hitters,0


In [7]:
# Train/Test Split
train = df[df['year'] < 2023]
test  = df[df['year'] >= 2023]

X_train = train[features]
y_train = train['woba']

X_test = test[features]
y_test = test['woba']


In [9]:
# Fit XGBoost & Predict Counterfactual
model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)
model.fit(X_train, y_train)

# Counterfactual predictions for post-ban (as if shift still existed)
test['pred_woba'] = model.predict(X_test)
test['lift'] = test['woba'] - test['pred_woba']

# RMSE for diagnostic
rmse = root_mean_squared_error(y_test, test['pred_woba'])
print(f"Post-Ban RMSE: {rmse:.4f}")


Post-Ban RMSE: 0.0084


C:\Users\Andrew\AppData\Local\Temp\ipykernel_4352\1498942010.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_woba'] = model.predict(X_test)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_4352\1498942010.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['lift'] = test['woba'] - test['pred_woba']
